In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [3]:
import pandas as pd
import pyproj

In [4]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [5]:
from pathlib import Path

In [6]:
import joblib

In [7]:
from mt_function_yl import *

In [8]:
import seaborn as sn

In [9]:
from scipy import stats

In [10]:
from tabulate import tabulate

In [11]:
import statsmodels.api as sm

### Data loading

In [12]:
# Traffic Count bicycle
btc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2019.csv')
btc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2020.csv')
btc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2021.csv')
# btc22
btc22 = pd.read_csv('/home/yelhe/script/mt/output/bic_tc_2022_new.csv')

# Traffic count car
vtc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2019.csv')
vtc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2020.csv')
vtc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2021.csv')
# vtc22
vtc22 = pd.read_csv('/home/yelhe/script/mt/output/car_tc_2022.csv')

In [13]:
btc22_0 = btc22.drop(['FK_STANDORT'], axis = 1)
btc22_0 = btc22_0.rename(columns = {'velo':'bicycle_traffic_count',
                                    'fk_zaehler':'FK_ZAEHLER'})

In [14]:
btc = pd.concat([btc19, btc20, btc21, btc22_0])
# btc1921 = pd.concat([btc19, btc20, btc21])
# vtc1921 = pd.concat([vtc19, vtc20, vtc21])

In [15]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7946 entries, 0 to 2169
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FK_ZAEHLER             7946 non-null   object 
 1   week                   7946 non-null   int64  
 2   bicycle_traffic_count  7946 non-null   float64
 3   month                  7946 non-null   int64  
 4   date                   5776 non-null   object 
 5   year                   7946 non-null   int64  
 6   OST                    5776 non-null   float64
 7   NORD                   5776 non-null   float64
 8   bicycle_traffic_norm   5776 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 620.8+ KB


In [16]:
vtc22_0 = vtc22.rename(columns = {'AnzFahrzeuge':'car_traffic_count'})

In [17]:
vtc = pd.concat([vtc19, vtc20, vtc21, vtc22_0])

In [18]:
vtc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40239 entries, 0 to 11465
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MSID               40239 non-null  object 
 1   week               40239 non-null  int64  
 2   car_traffic_count  40239 non-null  float64
 3   month              40239 non-null  int64  
 4   date               28773 non-null  object 
 5   year               40239 non-null  int64  
 6   OST                28773 non-null  float64
 7   NORD               28773 non-null  float64
 8   car_traffic_norm   28773 non-null  float64
dtypes: float64(4), int64(3), object(2)
memory usage: 3.1+ MB


In [72]:
# ac_btc = atime.merge(btc, how = 'left', on = ['year', 'month', 'week'])
# ac_vtc = atime.merge(vtc, how = 'left', on = ['year', 'month', 'week'])

In [19]:
# Shortest distance file
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_bicyclecount.csv') 
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_carcount.csv') 

In [81]:
raw = pd.read_csv('/home/yelhe/data/Traffic_network_variables/random_pseudo_new.csv')

In [82]:
raw = raw.reset_index()

In [85]:
raw = raw[['rand_point', 'index']]

In [87]:
raw = raw.rename(columns = {'index':'id'})

In [181]:
raw.to_csv('/home/yelhe/data/Traffic_network_variables/idconversion.csv', index = False)

In [88]:
rp_koord = pd.read_csv('/home/yelhe/script/mt/output/rp_koord.csv')

In [94]:
ranp = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ran_pseudo_new_withgeo.csv')

In [95]:
ranp = ranp[['rand_point', 'x', 'y']]

In [96]:
ranp = ranp.rename(columns = {'x':'xcoord','y':'ycoord'})
ranp = ranp.assign(lat = None, lon = None)
transformer = pyproj.Transformer.from_crs("EPSG:2056", "EPSG:4326")
# a latitude; b longitude
a, b = transformer.transform(ranp[['xcoord']], ranp[['ycoord']])
# longitude 经度, latitude 纬度
ranp = ranp.assign(lon = b, lat = a)
ranp.head()

,rand_point,xcoord,ycoord,lat,lon
0,59,2.684201e+06,1.248218e+06,47.379413,8.553632
1,90,2.684170e+06,1.245414e+06,47.354203,8.552691
2,117,2.684125e+06,1.245559e+06,47.355511,8.552124
3,133,2.684705e+06,1.245080e+06,47.351133,8.559713
4,162,2.684119e+06,1.249244e+06,47.388656,8.552737


In [97]:
df1 = pd.merge(raw, ranp, on = ['rand_point'])
df2 = pd.merge(df1, rp_koord, on = ['id'])

In [100]:
raw

,rand_point,id
0,59,0
1,90,1
2,117,2
3,133,3
4,162,4
...,...,...
990,16869,990
991,16871,991
992,16878,992
993,16906,993


In [104]:
# Shortest distance file
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_bicyclecount.csv') 
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_carcount.csv') 

In [106]:
disb = pd.merge(raw, disb, how = 'left', on = ['rand_point'])
disv = pd.merge(raw, disv, how = 'left', on = ['rand_point'])

In [108]:
disbu = disb[['id', 'fk_zaehler', 'distance']]
disbu = disbu.drop_duplicates()

In [109]:
disvu = disv[['id', 'MSID', 'distance']]
disvu = disvu.drop_duplicates()

In [132]:
vtc

,MSID,week,car_traffic_count,month,date,year,OST,NORD,car_traffic_norm
0,Z001M001,1,30246.0,1,2019-01-01,2019,2683009.89,1243936.2,-0.577408
1,Z001M001,2,42921.0,1,2019-01-08,2019,2683009.89,1243936.2,-0.255275
2,Z001M001,3,44804.0,1,2019-01-15,2019,2683009.89,1243936.2,-0.207419
3,Z001M001,4,44542.0,1,2019-01-22,2019,2683009.89,1243936.2,-0.214078
4,Z001M001,5,45872.0,1,2019-01-29,2019,2683009.89,1243936.2,-0.180276
...,...,...,...,...,...,...,...,...,...
11461,Z097M002,48,37068.0,12,NaN,2022,NaN,NaN,NaN
11462,Z097M002,49,65870.0,12,NaN,2022,NaN,NaN,NaN
11463,Z097M002,50,64623.0,12,NaN,2022,NaN,NaN,NaN
11464,Z097M002,51,60259.0,12,NaN,2022,NaN,NaN,NaN


In [111]:
disbu = disbu.rename(columns = {'fk_zaehler': 'FK_ZAEHLER'})

In [151]:
btcn = btc[['FK_ZAEHLER','bicycle_traffic_count']]

In [152]:
btcn = btcn.dropna()
btcn = btcn.loc[btcn['bicycle_traffic_count'] != 0]

In [154]:
btcm = btcn.groupby(['FK_ZAEHLER'])['bicycle_traffic_count'].mean().reset_index()

In [133]:
vtcn = vtc[['MSID','car_traffic_count']]
vtcn = vtcn.dropna()
vtcn = vtcn.loc[vtcn['car_traffic_count'] != 0]

In [135]:
vtcm = vtcn.groupby(['MSID'])['car_traffic_count'].mean().reset_index()

In [157]:
rp_bc = btcm.merge(disbu, how = 'left', on = ['FK_ZAEHLER'])
rp_vc = vtcm.merge(disvu, how = 'left', on = ['MSID'])

In [160]:
rp_bcr = rp_bc.dropna(subset = ['id'])

In [162]:
rp_vcr = rp_vc.dropna(subset = ['id'])

In [174]:
len(rp_bcr['id'].unique())

932

In [163]:
rp_vcr

,MSID,car_traffic_count,id,distance
0,Z001M001,44779.000000,538.0,407.922606
1,Z001M001,44779.000000,539.0,355.466381
2,Z001M001,44779.000000,571.0,704.535162
3,Z001M001,44779.000000,589.0,579.578427
4,Z001M001,44779.000000,590.0,184.857237
...,...,...,...,...
1022,Z096M004,17852.702857,72.0,779.892750
1023,Z096M004,17852.702857,74.0,715.152602
1024,Z096M004,17852.702857,124.0,594.261833
1026,Z097M002,60389.633484,293.0,222.848907


In [140]:
# print(len(rp_bc['id'].unique()), len(rp_vc['id'].unique()),
#       len(disvu['id'].unique()),len(disbu['id'].unique()))

995 995 995 995


In [164]:
rp_bcr.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_rp.csv')
rp_vcr.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_rp.csv')

In [165]:
rp_bcrm = rp_bcr.groupby(['id', 'distance'])['bicycle_traffic_count'].mean().reset_index()

In [175]:
rp_vcrm = rp_vcr.groupby(['id', 'distance'])['car_traffic_count'].mean().reset_index()

In [178]:
rp_bcrm.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecountm_rp.csv')
rp_vcrm.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcountm_rp.csv')